### <font color = 'navy'>Automate collecting rental flats information on HDB rental website</font>
### <font color = 'green'>Code Create Date: Sep-2021 </font>

#### <font>This code helps to automate in collecting several tables of HDB rental information.
#### This comes handy when someone wishes to get rental information from multiple blocks of flats in the same street.</font>

In [4]:
import pandas as pd
from selenium import webdriver
# https://selenium-python.readthedocs.io/

from webdriver_manager.chrome import ChromeDriverManager 
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select # for selecting dropdown list
import time

In [9]:
#initializing chrome by making an object of it
driver=webdriver.Chrome(ChromeDriverManager().install()) 
driver.maximize_window() #Maximize the window screen
url = "https://services2.hdb.gov.sg/webapp/BR12AWRentalEnq/BR12PSearch.jsp"
driver.get(url) 
time.sleep(1) #Wait for 1sec

############################################
blk_lists = ['79a', '79b','79c','79d','79e']
street_name = 'Toa Payoh Central'
############################################

table = list() # empty list collect rows of info
columnNames = ['Rental commence mth',
                'Town',
                'Blk no',
                'Street Name',
                'Flat Type',
                'Mnthly Rent (S$)']

for blk_list in blk_lists:
    
    # click the 'Block / Street' radio button
    driver.find_element(By.XPATH,
                '//*[@id="panelContent"]/div/div[3]/fieldset/div[2]/label[2]').click()
    driver.implicitly_wait(1) #Wait for 1sec
    
    # key blk number into the textbox
    driver.find_element(By.XPATH,'//*[@id="txtBlk"]').send_keys(blk_list) 
    driver.implicitly_wait(1) #Wait for 1sec
    
    # click the 'Get Street' button
    driver.find_element(By.XPATH,'//*[@id="btnRetrieveStreet"]').click() 
    driver.implicitly_wait(1) #Wait for 1sec
    
    # Select street name from dropdown list
    select = Select(driver.find_element(By.XPATH,'//*[@id="selStreet"]')) 
    select.select_by_visible_text(street_name)
    
    # click the 'from' calendar button
    driver.find_element(By.XPATH,
                '//*[@id="panelSearchByBlk"]/div[4]/div[2]/div[1]/div[1]/div/div[2]/a').click()
    driver.implicitly_wait(1) #Wait for 1sec

    # click '<' arrow to year 2020
    driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/table/thead/tr/th[1]/i').click() 
    driver.implicitly_wait(1) #Wait for 1sec
    
    #click October-2020
    driver.find_element(By.XPATH,'/html/body/div[3]/div[2]/table/tbody/tr/td/span[10]').click() 
    driver.implicitly_wait(1) #Wait for 1sec
    
    # click the 'to' calendar button
    driver.find_element(By.XPATH,
                '//*[@id="panelSearchByBlk"]/div[4]/div[2]/div[1]/div[3]/div/div[2]/a').click()
    driver.implicitly_wait(1) #Wait for 1sec
    
    #click September-2021
    driver.find_element(By.XPATH,'/html/body/div[4]/div[2]/table/tbody/tr/td/span[9]').click() 
    driver.implicitly_wait(1) #Wait for 1sec
    
    # click 'I Accept' button
    driver.find_element(By.XPATH,'//*[@id="btnSearch"]').click() 
    driver.implicitly_wait(1) #Wait for 1sec

    # Xpath for the entire table
    tableData = driver.find_element_by_xpath('//*[@id="result-paginated"]/div/table') 
    # Get the number of rows of data becos no. of rows will vary
    row_nums = len(tableData.text.splitlines()) 

    # Note: pd.read_html(driver.page_source) good only for single table. 
    # This case there is need to append tables
    for row_num in range(1,row_nums):
        
        row_list = [] # create an empty list for every loop
        for col_num in range(1,len(columnNames)+1): 
            # There are 6 columns: 
            # Rental commence mth, town, blk no, Street name, Flat type, monthly rent
            get_indiv_data = driver.find_element_by_xpath(
                f'//*[@id="result-paginated"]/div/table/tbody/tr[{row_num}]/td[{col_num}]')
            row_list.append(get_indiv_data.text) # forming a row of rental info
            
        table.append(row_list)
        
     
    # if it is the last block, Do Not click 'New Enquiry' button and exit the for loop
    if blk_list != blk_lists[-1]:
        # click 'New Enquiry' button
        driver.find_element(By.XPATH,'//*[@id="btnNewEnquiry"]').click() 
        driver.implicitly_wait(1) #Wait for 1sec 
    else:
        exit # or driver.quit() ??

driver.quit() # quits the driver instance, closing every associated window
        
# Insert the column names into dataframe
df = pd.DataFrame(table, columns= columnNames)
df 



====== WebDriver manager ======
Current google-chrome version is 93.0.4577
Get LATEST driver version for 93.0.4577
Driver [C:\Users\TanCK\.wdm\drivers\chromedriver\win32\93.0.4577.63\chromedriver.exe] found in cache


,Rental commence mth,Town,Blk no,Street Name,Flat Type,Mnthly Rent (S$)
0,Aug 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2400.00
1,Aug 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2700.00
2,Jul 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2550.00
3,Jul 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2600.00
4,Jul 2021,Toa Payoh,79A,Toa Payoh Central,5-Rm,2800.00
5,Jul 2021,Toa Payoh,79A,Toa Payoh Central,5-Rm,2900.00
6,Jul 2021,Toa Payoh,79A,Toa Payoh Central,5-Rm,2900.00
7,Jun 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2450.00
8,May 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2200.00
9,May 2021,Toa Payoh,79A,Toa Payoh Central,4-Rm,2400.00
